In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import numpy as np
from google.colab import files
from io import BytesIO
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

TESTING_ENABLED = True
ANALYSIS_ENABLED = False

In [ ]:
def cov(mat):
    """
    mat should be a matrix whose rows are separate samples.
    """
    xbar = torch.zeros(mat.size()[1], device=device)
    for row in mat:
        xbar += row
    xbar /= mat.size()[0]
    matdelta = [row-xbar for row in mat]
    print(f"Squares of L2 norms of X-Xbar: {[torch.nansum(torch.square(row)) for row in matdelta]}")
    ans = []
    for row in matdelta:
        unsqueezed_row = row.unsqueeze(0)
        ans.append(torch.matmul(unsqueezed_row.t(), unsqueezed_row))
    return sum(ans)/(len(ans)-1)

def get_eig_of_differences(data):
    """
    data should be a 2-dimensional tensor whose rows are the trained parameters
    of an ML model, with one training per row. If you have a model with N
    parameters that you run M times, data should be of size (M, N). Returns a
    1-dimensional, M-element tensor whose entries are the eigenvalues (in no
    particular order) of the square matrix calculated by subtracting the mean
    of the input rows from each individual row, then right-multiplying that
    matrix by its transpose. This guarantees a M by M square matrix, which
    means that calculating the eigenvalues is surprisingly fast, even with
    reasonably large models.
    """
    mu = torch.mean(data, 0, keepdim=True).expand(data.size()[0], -1)
    deviation_data = data-mu
    square_data = torch.matmul(deviation_data, deviation_data.transpose(0, 1))
    eig = torch.linalg.eigvals(square_data)
    return eig.real

def get_l2_norm(data):
    mu = torch.mean(data, 0, keepdim=True).expand(data.size()[0], -1)
    deviation_data = data-mu
    total_norm = 0.0
    for row in deviation_data:
        norm = torch.norm(row, p=2)
        norm *= norm
        total_norm += norm
    return total_norm / data.size()[0]

def get_sample_covariance_eigs(data):
    cov = torch.cov(data.t()) # torch.cov expects columns, not rows, to be individual trials
    eig = torch.linalg.eigvals(cov)
    return eig.real

def get_eig_of_differences_fast(data, num_eigvals=-1):
    """
    ONLY USE THIS FUNCTION AFTER RUNNING AN EXCESSIVE (>10,000) NUMBER OF TRIALS
    Acts similarly to get_eig_of_differences, except that it averages the N by N
    square matrix whose eigenvalues are calculated with its transpose, then uses
    torch.lobpcg to estimate only the num_eigvals largest eigenvalues, instead
    of calculating all of them. num_eigvals defaults to data.size()[0]
    """
    mu = torch.mean(data, 0, keepdim=True).expand(data.size()[0], -1)
    deviation_data = data-mu
    square_data = torch.matmul(deviation_data, deviation_data.transpose(0, 1))
    square_data = (square_data+square_data.transpose(0, 1))/2
    eigvals_to_gen = data.size()[0] if num_eigvals == -1 else num_eigvals
    eig = torch.lobpcg(square_data, k=eigvals_to_gen)
    return eig[0]

def generate_gaussian_noise(cov, noise_mult=1):
    U, S, V = torch.linalg.svd(cov)
    plt.hist([eig.item() for eig in S], bins=50, log=True)
    print(f"Sum of eigenvalues: {torch.nansum(S)}")
    Sprime = torch.sqrt(S)
    print(f"sqrt sum of eigenvalues: {torch.nansum(Sprime)}")
    Sprime *= torch.nansum(Sprime)
    Sprime = torch.sqrt(Sprime)
    print(f"Average eigenvalue of noise (after sqrt etc.): {sum(Sprime)/Sprime.size()[0]}. Max after-processing eigenvalue of noise: {torch.max(Sprime)}")
    plt.hist([eig.item() for eig in Sprime], bins=50, log=True)
    print(sum(U[0]), sum(V[0]))
    print(f"L2 norm of noise (before being multiplied by {noise_mult}): {math.sqrt(sum(torch.square(torch.matmul(U, torch.matmul(torch.ones(cov.size()[0], device=device), torch.diag(Sprime))))))}")
    noise = torch.randn(cov.size()[0], device=device)*noise_mult
    noise = torch.matmul(noise, torch.diag(Sprime))
    noise = torch.matmul(U, noise)
    return noise

In [ ]:
print("Please upload a matrix to analyze. Each row should be the results of one trial.")
uploaded = files.upload()
if len(uploaded.keys()) == 1:
    data = torch.load(BytesIO(list(uploaded.values())[0]), map_location=device)
else:
    data = []
    for trial in uploaded.values():
        data.append(torch.load(BytesIO(trial, map_location=device)))
    if len(data[0].size()) == 1:
        data = torch.stack(data)
    else:
        data = torch.cat(data, 0)
assert isinstance(data, torch.Tensor)
assert len(data.size()) == 2
data = data.to(device)
print(f"input matrix successfully loaded. It is of size {data.size()}")
if ANALYSIS_ENABLED:
    eig = get_sample_covariance_eigs(data)
    eig.to('cpu')
    print(f"Eigenvalues: {eig}")
    norm = get_l2_norm(data)
    print(f"L2 norm: {norm}")
    torch.save(eig, "SampleCovarianceEigenvalues.pt")
    files.download("SampleCovarianceEigenvalues.pt")
    print(f"Sum of sqrts of eigenvalues: {torch.nansum(torch.sqrt(eig))}")

In [ ]:
def _weights_init(m):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def resnet20():
    return ResNet(BasicBlock, [3, 3, 3]).to(device)

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD_DEV = (0.2023, 0.1994, 0.2010)

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD_DEV),
])
testset = torchvision.datasets.CIFAR10(root='./data', train = False, download=True, transform=transform)

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

def test_model(model, preprocessing = lambda x: x):
    test_loss = 0.0
    testloader = torch.utils.data.DataLoader(testset, shuffle=True, num_workers=2, batch_size = 20)
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    train_on_gpu = torch.cuda.is_available()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    model.eval()
    # iterate over test data
    for data, target in testloader:
        # move tensors to GPU if CUDA is available
        data, target = data.to(device), target.to(device)
        # preprocess data with an arbitrary function
        data = preprocessing(data)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update test loss
        test_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(20):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
    # average test loss
    test_loss = test_loss/len(testloader.dataset)
    print('Test Loss: {:.6f}\n'.format(test_loss))

    for i in range(10):
        if class_total[i] > 0:

            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
              classes[i], 100 * class_correct[i] / class_total[i],
              np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))
    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))
    return 100. * np.sum(class_correct) / np.sum(class_total)

class DenseModel(nn.Module):

    def __init__(self):
        super(DenseModel, self).__init__()
        self.layer1 = nn.Linear(64, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 32)
        self.layer4 = nn.Linear(32, 16)
        self.layer5 = nn.Linear(16, 16)
        self.layer6 = nn.Linear(16, 10)

    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = F.relu(self.layer2(out))
        out = F.relu(self.layer3(out))
        out = F.relu(self.layer4(out))
        out = F.relu(self.layer5(out))
        out = self.layer6(out)
        return out

def load_params(model, params):
    """
    This will error (or possibly silenetly fail!) if params were not generated
    using the get_params function from a model of the same type as the model
    passed in.
    """
    params_idx = 0
    with torch.no_grad():
        for param in model.parameters():
            params_needed = param.flatten().size()[0]
            params_to_take = params[params_idx:params_idx+params_needed]
            params_idx += params_needed
            param.copy_(params_to_take.reshape(param.size()))
    return model

In [ ]:
MODEL_IDX = 24 # The number of the model we are using. Should not be greater than the number of trials that have been run
NOISE_MULTIPLIER = 1 # TODO determine what levels of noise correspond to what levels of DP (FAR easier said than done)

if TESTING_ENABLED:
    preprocessing = resnet20()
    preprocessing.to(device)
    print("Please upload the pretrained ResNet20 used when training the analyzed models.")
    uploaded = files.upload()
    preprocessing.load_state_dict(torch.load(BytesIO(list(uploaded.values())[0]), map_location=device))
    preprocessing.linear = Identity()
    preprocessing.eval()

    model = DenseModel()
    model.to(device)
    params = data[MODEL_IDX].clone().detach()
    print(f"L2 norm of model parameters: {math.sqrt(sum(torch.square(params)))}")
    load_params(model, params)

    print("Testing model without noise:")
    test_model(model, preprocessing)

    noise = generate_gaussian_noise(cov(data), NOISE_MULTIPLIER)
    with torch.no_grad():
        params += noise
        load_params(model, params)

    print(f"Testing model with noise scaled by a factor of {NOISE_MULTIPLIER}")
    test_model(model, preprocessing)
